In [15]:
import os
import boto3

import pandas as pd
import numpy as np
from sagemaker.session import Session

In [5]:
# initiate s3 bucket and corresponding data folder
# bucket = "ran-s3-systemic-risk"
# data_folder ="Input/X-17A-5/"

bucket = "ran-s3-systemic-risk"
data_folder ="Output/BalanceSheet/"

# Amazon Textract client and Sagemaker session
textract = boto3.client('textract')
s3 = boto3.client('s3')
session = Session()

# discover all of the pdfs that you want to parse
paths = np.array(session.list_s3_files(bucket, data_folder))

In [12]:
selections = np.random.choice(paths[1:], 100)

In [33]:
assetNames = []

In [34]:
for file in selections:

    # retrieving downloaded files from s3 bucket
    s3.download_file(bucket, file, 'temp.pdf')
    
    df = pd.read_csv('temp.pdf')
    names = df[df.columns[1]].dropna().values
    assetNames.append(names)
    
    # remove local file after it has been created
    os.remove('temp.pdf')

In [37]:
assetNames[:2]

[array(['Cash', 'Deposit with clearing broker', 'Trade receivables',
        'Securities owned, at market value', 'Due from Parent',
        'Furniture, equipment, and leasehold improvements (net of accumulated depreciation and amortization of $100,436)',
        'Other assets', "Liabilities and stockholder's equity",
        'Accounts payable and accrued liabilities',
        'Obligation under capital lease', "Stockholder's equity:",
        'Common stock without par value, $10 stated value,',
        '100,000 authorized shares, 95,500 shares issued;',
        '71,965 shares outstanding', 'Additional paid-in capital',
        'Retained earnings',
        'Less: Treasury stock, at cost (23,535 shares)',
        "Total stockholder's equity"], dtype=object),
 array(['Cash', 'Securities owned, at market value',
        'Due from clearing broker', 'Notes receivable from Parent',
        'Due from Parent and other related parties',
        'Furniture, equipment and leasehold improvements',


In [40]:
def intersection(mat:np.ndarray) -> np.ndarray:
# Finds the intersection of all array within the matrix
# :param: mat (np.array)

    interArr = mat[0]
    # itterate through the asset names 
    for i in range(len(mat)-1):
        # find the intersection amongst all array
        interArr = np.intersect1d(interArr, mat[i+1])
        
    return interArr

In [38]:
np.intersect1d(assetNames[1], assetNames[2])

array(['Additional paid-in capital', 'Cash', 'Liabilities:',
       'TOTAL ASSETS'], dtype=object)

In [47]:
intersection(assetNames[42:43])

array(['ASSETS -', 'Cash and cash equivalents', 'Receivables:',
       'Clearing brokers', 'Commissions', 'Other',
       'Deposits with clearing brokers',
       'Securities owned, at market value', 'Other assets', '-',
       "LIABILITIES AND SHAREHOLDER'S EQUITY", 'LIABILITIES:',
       'Commissions payable', 'Accounts payable',
       'Due to clearing broker',
       'Securities sold, but not yet purchased, at market value',
       'Due to affiliates (Note 3)', 'TOTAL LIABILITIES',
       'CONTINGENCIES (Note 4)', "SHAREHOLDER'S EQUITY (Note 2):",
       'Common stock, $.01 par value; 750,000 shares authorized;',
       '10,000 shares issued and outstanding',
       'Additional paid-in capital', 'Retained earnings',
       "TOTAL SHAREHOLDER'S EQUITY"], dtype=object)